In [2]:
from AOC_utils import get_day
import numpy as np

day = 5
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 5 input already downloaded
1371


['84|39', '48|11', '48|79', '88|98', '88|82', '88|12', '69|67']

In [3]:
# part 1
example = '''
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
'''.split('\n')[1:-1]

In [31]:
def solve_day(data):
    part1 = 0
    part2 = 0

    page_orders = []
    updates = []

    for line in data:
        if '|' in line:
            page_order = line.split('|')
            page_orders.append(page_order)
        elif ',' in line:
            update = line.split(',')
            updates.append(update)

    page_mapping = {}
    for page_order in page_orders:
        if page_order[0] not in page_mapping:
            page_mapping[page_order[0]] = [page_order[1]]
        else:
            page_mapping[page_order[0]] += [page_order[1]]

    for update in updates:
        old_update = update

        updatebroken = False
        
        for i in range(len(update)):
            n = update[i]
            if n in page_mapping:
                mapping = page_mapping[n]
                intersection = np.intersect1d(update[:i], mapping)

                if len(intersection) != 0:
                    updatebroken = True

                    indexes = [update.index(i) for i in intersection]
                    first_intersection = min(indexes)

                    update.pop(i)
                    update.insert(first_intersection, n)
                    
        if not updatebroken:
            part1 += int(old_update[(len(old_update)//2)])
        else:
            part2 += int(update[(len(update)//2)])

    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example)

part 1: 143
part 2: 123


In [32]:
solve_day(input_data)

part 1: 4905
part 2: 6204
